In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Tue Jul  7 16:46:16 PDT 2020


In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/ethiopia/2020_06_28_12_40_56/count_data/']

locations = ['Ethiopia']

# 1. Iron effect on birth weight

In [4]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

# note: no stratification of birth counts by iron coverage group

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,21,baseline,0.0
1,2020,female,covered,live_births,21,baseline,0.0
2,2020,female,covered,live_births,21,baseline,0.0
3,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,0.0
4,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,0.0


In [5]:
bw = pd.read_hdf(output_dirs[0] + 'birth_weight.hdf')
bw.head()

,year,sex,measure,input_draw,scenario,value,iron_fortification_group
0,2020,female,birth_weight_mean,21,baseline,3375.170761,uncovered
1,2020,female,birth_weight_mean,21,baseline,3150.741892,covered
2,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3375.170761,uncovered
3,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3150.741892,covered
4,2020,female,birth_weight_mean,21,iron_folic_acid_fortification_scale_up,3375.170761,uncovered


In [6]:
bw.loc[bw.measure == 'birth_weight_mean'].groupby(['scenario','iron_fortification_group','year']).mean()

input_draw  \
scenario                               iron_fortification_group year               
baseline                               covered                  2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
                                       uncovered                2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
folic_acid_fortification_scale_up      covered                  2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
                                       uncovered                2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
iron_folic_acid_fortification_scale_up covered                  2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
                                       uncovered                2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
vitamin_a_fortification_scale_up       covered                  2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   
                                       uncovered                2020      437.52   
                                                                2021      437.52   
                                                                2022      437.52   
                                                                2023      437.52   

                                                                            value  
scenario                               iron_fortification_group year               
baseline                               covered                  2020   960.860742  
                                                                2021   953.329325  
                                                                2022   938.893509  
                                                                2023   932.639694  
                                       uncovered                2020  3370.576127  
                                                                2021  3370.571867  
                                                                2022  3369.317002  
                                                                2023  3370.021562  
folic_acid_fortification_scale_up      covered                  2020   960.860742  
                                                                2021   953.329325  
                                                                2022   938.893509  
                                                                2023   932.639694  
                                     

In [7]:
bw_by_coverage_and_scenario = bw.loc[bw.measure == 'birth_weight_mean']
bw_by_coverage_and_scenario = bw_by_coverage_and_scenario.groupby(['scenario','iron_fortification_group']).mean()
bw_by_coverage_and_scenario

# we would expect mean birth weight to be higher in covered group than uncovered group (for all scenarios)
# we are seeing the opposite
# the birth weights in the covered and uncovered group have way too much separation (we would expect close to 10 g difference)

input_draw  \
scenario                               iron_fortification_group               
baseline                               covered                       437.52   
                                       uncovered                     437.52   
folic_acid_fortification_scale_up      covered                       437.52   
                                       uncovered                     437.52   
iron_folic_acid_fortification_scale_up covered                       437.52   
                                       uncovered                     437.52   
vitamin_a_fortification_scale_up       covered                       437.52   
                                       uncovered                     437.52   

                                                                       value  
scenario                               iron_fortification_group               
baseline                               covered                    946.430818  
                                       uncovered                 3370.121640  
folic_acid_fortification_scale_up      covered                    946.430818  
                                       uncovered                 3370.121640  
iron_folic_acid_fortification_scale_up covered                   2585.263280  
                                       uncovered                 3370.291160  
vitamin_a_fortification_scale_up       covered                    946.430818  
                                       uncovered                 3370.121640

## Birth weight conclusions

Birth weight is higher in the uncovered group - opposite as expected

Birth weight separation between coverage groups is WAY too high (it is currently ~2500 g, should be ~10)

# 2. Vitamin A Fortification and Vitamin A Deficiency Prevalence

In [8]:
state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
state_pt.head()

,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
1,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
2,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
3,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
4,1_to_4,diarrheal_diseases,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0


In [3]:
def calculate_stratified_vad_prevalence(strata_cols):
    state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    vad = state_pt.loc[state_pt.cause == 'vitamin_a_deficiency']
    vad = vad.groupby((['scenario','input_draw'] + strata_cols)).sum()
    pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    pt = pt.loc[pt.cause.str.contains('diarrheal')]
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    vad_prev = vad / pt
    vad_prev = vad_prev.reset_index()
    vad_prev = vad_prev.loc[vad_prev.scenario != 'iron_fortification_scale_up']
    vad_prev = (vad_prev.groupby((['scenario'] + strata_cols)).mean()
                .rename(columns={'value':'vad_prevalence'})
                .drop(columns='input_draw'))
    return vad_prev

In [10]:
overall = calculate_stratified_vad_prevalence([])
overall

# looks good! (Prevalence of VAD is lower in vitamin_a_fortification_scale_up scenario than baseline scenario)

,vad_prevalence
scenario,
baseline,0.366547
folic_acid_fortification_scale_up,0.366547
iron_folic_acid_fortification_scale_up,0.366547
vitamin_a_fortification_scale_up,0.329076


In [ ]:
#by_year = calculate_stratified_vad_prevalence(['year'])
#by_year

# can't currently check with Ethiopia count space data not stratified by year

In [4]:
by_age = calculate_stratified_vad_prevalence(['age_group'])
by_age

# no change in neonatal age groups between scenarios, as expected
# lower VAD prevalence in vitamin_a_scenario in post_neonatal and 1_to_4 ages, as expected
# looks good!

vad_prevalence
scenario                               age_group                     
baseline                               1_to_4                0.317167
                                       early_neonatal        0.846027
                                       late_neonatal         0.809661
                                       post_neonatal         0.531507
folic_acid_fortification_scale_up      1_to_4                0.317167
                                       early_neonatal        0.846027
                                       late_neonatal         0.809662
                                       post_neonatal         0.531506
iron_folic_acid_fortification_scale_up 1_to_4                0.317167
                                       early_neonatal        0.846027
                                       late_neonatal         0.809662
                                       post_neonatal         0.531506
vitamin_a_fortification_scale_up       1_to_4                0.278145
                                       early_neonatal        0.846027
                                       late_neonatal         0.809661
                                       post_neonatal         0.497268

In [5]:
by_coverage_group = calculate_stratified_vad_prevalence(['vitamin_a_fortification_group'])
by_coverage_group

# unexpected results here...

# we should expect...

    # all VAD_prev < 1 in each coverage group -- WE SEE THIS!
          
    # effectively_covered VAD_prev * 2.22 ~= uncovered VAD_prev in both scenarios
        # the difference we see is slightly less
        
    # all VAD_prev by coverage group in baseline scenario should ~= VAD_prev by coverage group in vitamin_a scenario
        # they are slightly off, but close
        
    # note: covered VAD prev > uncovered VAD prev because covered is mostly neonatal age groups, which have higher PREV

vad_prevalence
scenario                               vitamin_a_fortification_group                
baseline                               covered                              0.551638
                                       effectively_covered                  0.204428
                                       uncovered                            0.369196
folic_acid_fortification_scale_up      covered                              0.551638
                                       effectively_covered                  0.204428
                                       uncovered                            0.369196
iron_folic_acid_fortification_scale_up covered                              0.551638
                                       effectively_covered                  0.204428
                                       uncovered                            0.369196
vitamin_a_fortification_scale_up       covered                              0.460052
                                       effectively_covered                  0.193830
                                       uncovered                            0.369572

## Conclusions

- This looks good!

# 3. Iron effect on hemoglobin

In [6]:
hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
hb.head()

,sex,measure,input_draw,scenario,value,age,status,responsive
0,female,hemoglobin_mean,21,baseline,92.417444,0.5,covered,responsive
1,female,hemoglobin_mean,21,baseline,17.384198,0.5,covered,non-responsive
2,female,hemoglobin_mean,21,baseline,99.709819,0.5,uncovered,responsive
3,female,hemoglobin_mean,21,baseline,91.963281,0.5,uncovered,non-responsive
4,female,hemoglobin_mean,21,baseline,97.879825,1,covered,responsive


In [7]:
np.unique(hb['measure'])

array(['hemoglobin_mean', 'hemoglobin_variance'], dtype=object)

In [8]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

# NOTE: cannot calculate weighted averages of Hb because persontime data not stratified by iron coverage or responsiveness

,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,1_to_4,covered,covered,person_time,21,baseline,0.0
1,1_to_4,covered,covered,person_time,21,baseline,0.0
2,1_to_4,covered,covered,person_time,21,baseline,0.0
3,1_to_4,covered,covered,person_time,21,baseline,0.0
4,1_to_4,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0


In [9]:
def get_stratified_hb_mean(strata_cols):
    hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
    hb = hb.loc[hb.scenario != 'vitamin_a_fortification_scale_up']
    hb = hb.loc[hb.measure == 'hemoglobin_mean']
    hb = hb.groupby(['scenario','measure'] + strata_cols).mean()
    return hb

In [10]:
overall = get_stratified_hb_mean([])
overall

# we would expect to see much smaller increase in mean hemoglobin (less than 3 units)

,,input_draw,value
scenario,measure,,
baseline,hemoglobin_mean,437.52,59.652004
folic_acid_fortification_scale_up,hemoglobin_mean,437.52,59.652085
iron_folic_acid_fortification_scale_up,hemoglobin_mean,437.52,86.634593


In [11]:
by_coverage = get_stratified_hb_mean(['status'])
by_coverage

# hemoglobin is lower in covered group, which is the opposite of what we would expect
# difference between covered and uncovered groups is way too high (should be <3)
# should not be such a difference between covered group in iron scenario compared to baseline scenario
    # iron scenario should be slighlty LESS than baseline scenario

# should expect...
    
    # covered mean hemoglobin to be higher than uncovered mean hemoglobin in both scenarios
        # note, if this is not true, it should be true for MEDIAN hemoglobin by coverage group
    # mean hemoglobin by coverage group should be approximately equal between scenarios
        # note, it is ok if covered hemoglobin in the iron scenario is slightly less than the covered 
        # group in the baseline scenario

input_draw  \
scenario                               measure         status                  
baseline                               hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   

                                                                      value  
scenario                               measure         status                
baseline                               hemoglobin_mean covered    20.463565  
                                                       uncovered  98.840442  
folic_acid_fortification_scale_up      hemoglobin_mean covered    20.463731  
                                                       uncovered  98.840440  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered    74.470406  
                                                       uncovered  98.798779

In [12]:
by_responsiveness = get_stratified_hb_mean(['status','responsive'])
by_responsiveness

# we are unexpected results here

    
# expected results:

    # for a given scenario...
        # covered, responsive hemoglobin > uncovered, responsive hemoglobin (by ~ 3 units)
        # covered, non-responsive hemoglobin ~= uncovered, non-responsive hemoglobin
    
    # mean hemoglobin by coverage group and responsiveness should be approximately equal between scenarios
        
    # for a given coverage group, responsive hemoglobin > non-responsive hemoglobin
        # for a given ANEMIA group (i.e. severe anemia)... responsive hemoglobin ~= non-responsive hemoglobin
            # can verify this in interactive sim?
        # note: difference between responsive and non-responsive should be larger in covered group than uncovered group

input_draw  \
scenario                               measure         status    responsive                   
baseline                               hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   

                                                                                      value  
scenario                               measure         status    responsive                  
baseline                               hemoglobin_mean covered   non-responsive   11.110789  
                                                                 responsive       29.816342  
                                                       uncovered non-responsive   94.193903  
                                                                 responsive      103.486982  
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive   11.110789  
                                                                 responsive       29.816673  
                                                       uncovered non-responsive   94.193903  
                                                                 responsive      103.486976  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive   54.482087  
                                                                 responsive       94.458726  
                                                       uncovered non-responsive   94.111256  
                                                                 responsive      103.486303

## Conclusions

- we are seeing implausible hemoglobin values here (29). Potentially due to issues in coverage algorithm?
- hemoglobin is lower in covered group, which is the opposite of what we would expect. WAY too much separation
- there is significant variation in mean hemoglobin between responsive and non-responsive groups in the uncovered population, which is unexpected
- seeing an increase in mean hemoglobin between scenarios in the non-responsive population, which is unexpected

# 5. VAD and affected causes

In [13]:
deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
deaths.head()

,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
1,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
2,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
3,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
4,1_to_4,diarrheal_diseases,covered,covered,death,21,folic_acid_fortification_scale_up,0.0


In [14]:
np.unique(deaths['cause'])

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [15]:
# vad should affect LRI, measles, DD, but NOT NTDs

In [16]:
def get_csmrs(strata_cols):
    deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
    pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    pt = pt.loc[pt.cause.str.contains('diarrheal')]
    deaths = deaths.groupby((['scenario','input_draw'] + strata_cols + ['cause'])).sum()
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    csmrs = deaths / pt
    csmrs = csmrs.reset_index()
    csmrs = csmrs.loc[csmrs.scenario != 'iron_fortification_scale_up']
    csmrs = csmrs.groupby((['cause'] + strata_cols + ['scenario'])).mean()
    return csmrs

In [17]:
overall = get_csmrs([])
overall

# looks good!
# CSMR for other causes may be slightly higher in vitamin A scenario because 
# simulants who did not die of affected causes can now die of another cause

input_draw  \
cause                        scenario                                             
diarrheal_diseases           baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
measles                      baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
neural_tube_defects          baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
other_causes                 baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   

                                                                        value  
cause                        scenario                                          
diarrheal_diseases           baseline                                0.001918  
                             folic_acid_fortification_scale_up       0.001918  
                             iron_folic_acid_fortification_scale_up  0.001918  
                             vitamin_a_fortification_scale_up        0.001863  
lower_respiratory_infections baseline                                0.001286  
                             folic_acid_fortification_scale_up       0.001286  
                             iron_folic_acid_fortification_scale_up  0.001286  
                             vitamin_a_fortification_scale_up        0.001276  
measles                      baseline                                0.000286  
                             folic_acid_fortification_scale_up       0.000286  
                             iron_folic_acid_fortification_scale_up  0.000286  
                             vitamin_a_fortification_scale_up        0.000275  
neural_tube_defects          baseline                                0.000162  
                             folic_acid_fortification_scale_up       0.000158  
                             iron_folic_acid_fortification_scale_up  0.000158  
                             vitamin_a_fortification_scale_up        0.000162  
other_causes                 baseline                                0.007031  
                             folic_acid_fortification_scale_up       0.007031  
                             iron_folic_acid_fortification_scale_up  0.007030  
                             vitamin_a_fortification_scale_up        0.007029

In [18]:
by_age = get_csmrs(['age_group'])
by_age

# this is behaving as expected
# lower CSMRs for postneonatal and 1-4 age group, but not neonatal age groups

input_draw  \
cause                        age_group      scenario                                             
diarrheal_diseases           1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
...                                                                                        ...   
neural_tube_defects          1_to_4         iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortif

In [20]:
#by_year = get_csmrs(['year'])
#by_year

# this is behaving as we would expect with the CORRECT population coverage algorithm (no change until 2020)

## Conclusions

This is looking as expected (VAD prevalence appears to be affecting CSMRs due to diarrheal diseases, measles, and LRI.

However, it also appears to be affecting "other causes" which is unexpected